In [1]:
# default_exp codexplainer.mutual_info

In [232]:
# export

import dit
import numpy as np
import pandas as pd
from ds4se.mgmnt.prep.tokenization_counting import *
from ds4se.mgmnt.prep.bpe_tokenization import HFTokenizer

from typing import Optional, List, Tuple, Dict

from ds4se.mgmnt.prep.files_mgmnt import jsonl_to_dataframe

In [3]:
# export
#Logging configuration

import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formacompute_info_theory_measurester)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

### mutual_info explainer

> This module provides methods for performing comparisons between code samples leveraging information theory measurements

> @Alvaro


Common use leverage HuggingFace tokenizer

In [50]:
params = {
    "hf_tokenizer_path": "/home/jovyan/work/int-data/tokenizer/tokenizer-java-v1.json",
    "sample_path": "/home/jovyan/work/int-data/samples/generators/transformers/tfr_layers6_vocab10000_embd768_heads12/5000-samples_1625693099.45201.jsonl",
    "hmn_sample_path": "/home/jovyan/work/int-data/samples/human/train/5000-samples_1621479183.15598.jsonl"
}

In [5]:
hf_tokenizer = HFTokenizer(params["hf_tokenizer_path"])

In [12]:
java_samples = jsonl_to_dataframe(params["sample_path"])

2021-09-11 18:08:54,885 : INFO : Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-09-11 18:08:54,886 : INFO : Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-09-11 18:08:54,887 : INFO : NumExpr defaulting to 8 threads.


In [13]:
toks_frequencies = get_tokens_frequency_hf_tkzr(java_samples, hf_tokenizer)

In [239]:
# export

def get_mutual_info_hf(hf_tokenizer: HFTokenizer,
                       sample_set1: List[str],
                       sample_set2: List[str]) -> pd.DataFrame:
    """
    Perform mutual info. analysis for 2 sample sets based on token counting
    :param hf_tokenizer: HuggingFace tokenizer for performing counting
    :param sample_set1: List containing paths for sample set 1 paths
    :param sample_set2: List containing paths for sample set 2 paths
    
    :return: Dict containing data for
             - Mutual information
             - Entropy for sample set 1
             - Entropy for sample set 2
             - Entropy for joint distributions
             
    """
    vocab = list(hf_tokenizer.tokenizer.get_vocab().keys())
    # For numerical stability
    epsilon = 4.9406564584124654e-324
    
    if(len(sample_set1) != len(sample_set2)):
        logging.error("Sample sets have different sizes, cannot compare them.")
        return
    
    n_experiments = len(sample_set1)
    mutual_info_data = []
    entropy_s1_data = []
    entropy_s2_data = []
    joint_entropy_data = []
    
    for e in range(n_experiments):
        s1 = jsonl_to_dataframe(sample_set1[e])
        s2 = jsonl_to_dataframe(sample_set2[e])
        
        # Perform counting
        freqs_s1 = get_tokens_frequency_hf_tkzr(s1, hf_tokenizer)
        freqs_s2 = get_tokens_frequency_hf_tkzr(s2, hf_tokenizer)
        
        count_s1 = np.zeros(len(vocab))
        count_s2 = np.zeros(len(vocab))
        
        total_toks_s1 = 0
        total_toks_s2 = 0
        
        for i in range(len(vocab)):
            token = vocab[i]
            tk_c_s1 = freqs_s1[token] if token in freqs_s1 else epsilon
            tk_c_s2 = freqs_s2[token] if token in freqs_s2 else epsilon
            
            count_s1[i] = tk_c_s1
            count_s2[i] = tk_c_s2
            
            total_toks_s1 += tk_c_s1
            total_toks_s2 += tk_c_s2
            
        mutual_info, entropy_s1, entropy_s2, entropy_joint = compute_info_theory_measures(
            count_s1, count_s2,
            total_toks_s1, total_toks_s2,
            vocab
        )
        
        mutual_info_data.append(mutual_info)
        entropy_s1_data.append(entropy_s1)
        entropy_s2_data.append(entropy_s2)
        joint_entropy_data.append(entropy_joint)
    
    data = {
        "Mutual-info": mutual_info_data,
        "Entropy-S1": entropy_s1_data,
        "Entropy-S2": entropy_s2_data,
        "Entropy-joint": joint_entropy_data
    }
    
    return pd.DataFrame(data)

In [225]:
# export

def compute_info_theory_measures(vector_s1: np.ndarray, vector_s2: np.ndarray,
                                 tot_toks_s1: int, tot_toks_s2: int,
                                 vocab: List[str]) -> Tuple[float, float, float, float]:
    """
    Compute info. theory measures based on tokens count and vocabulary
    (Distributions are computed leveraging log_2)
    
    :param vector_s1: Vector containing counts for the entire vocab (sample set 1)
    :param vector_s2: Vector containing counts for the entire vocab (sample set 2)
    :param tot_toks_s1: int indicating total number of counted tokens for sample set 1
    :param tot_toks_s2: int indicating total number of counted tokens for sample set 2
    :param vocab: List containing the entire vocabulary
    
    :return: Tuple containing
             - Mutual info. calculation
             - Entropy for sample set 1 distribution 
             - Entropy for sample set 2 distribution 
             - Entropy for joint distribution
    """
    # Get individual distributions
    proportions_s1 = vector_s1 / tot_toks_s1
    proportions_s2 = vector_s2 / tot_toks_s2
    
    log_proportions_s1 = np.log2(proportions_s1)
    log_proportions_s2 = np.log2(proportions_s2)
    
    sum_p1 = np.sum(proportions_s1)
    sum_p2 = np.sum(proportions_s2)
    if sum_p1 != 1 or sum_p2 != 1:
        logging.warning("Vector don't correspond to distributions (don't add up to 1.0)")
        logging.warning("Vector sum for dist 1: {}".format(sum_p1))
        logging.warning("Vector sum for dist 2: {}".format(sum_p2))
    
    if len(proportions_s1) != len(vocab) or len(proportions_s2) != len(vocab):
        logging.error("Different lengths for vocab distributions")
        # return
    
    dist_s1 = dit.Distribution.from_ndarray(log_proportions_s1, base=2)
    dist_s2 = dit.Distribution.from_ndarray(log_proportions_s2, base=2)
    
    entropy_s1 = dit.shannon.entropy(dist_s1)
    entropy_s2 = dit.shannon.entropy(dist_s2)
    
    # Get joint distribution
    joint = (vector_s1 + vector_s2) / (tot_toks_s1 + tot_toks_s2)
    log_joint = np.log2(joint)
    joint_dist = dit.Distribution.from_ndarray(log_joint, base=2)
    entropy_joint = dit.shannon.entropy(joint_dist)
    
    # Get mutual information
    mutual_info = entropy_s1 + entropy_s2 - entropy_joint
    return mutual_info, entropy_s1, entropy_s2, entropy_joint

In [1]:
from nbdev.export import notebook2script
notebook2script()

Converted 0.1_mgmnt.prep.ipynb.
Converted 0.2_mgmnt.prep.files_mgmnt.ipynb.
Converted 0.3_mgmnt.prep.bpe_tokenization.ipynb.
Converted 0.4_mgmnt.prep.tokenization_counting.ipynb.
Converted 0.5_mgmnt.prep.token_mgmnt.ipynb.
Converted 1.1_exp.info.ipynb.
Converted 1.2_exp.desc.metrics.java.ipynb.
Converted 1.4_exp.metrics_python.ipynb.
Converted 1.5_exp.metrics_java.ipynb.
Converted 2.0_repr.codebert.ipynb.
Converted 2.0_repr.i.ipynb.
Converted 2.1_repr.codeberta.ipynb.
Converted 2.1_repr.roberta.train.ipynb.
Converted 2.2_repr.roberta.eval.ipynb.
Converted 2.3_repr.word2vec.train.ipynb.
Converted 2.6_repr.word2vec.eval.ipynb.
Converted 2.7_repr.distmetrics.ipynb.
Converted 2.8_repr.sentence_transformers.ipynb.
Converted 3.1_traceability.unsupervised.eda.ipynb.
Converted 3.2_traceability.unsupervised.approach.d2v.ipynb.
Converted 3.2_traceability.unsupervised.approach.w2v.ipynb.
Converted 4.0_infoxplainer.ir.ipynb.
Converted 4.1_infoxplainer.ir.unsupervised.d2v.ipynb.
Converted 4.2_infox